In [3]:
# Step 1: Import the necessary libraries

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd


In [4]:
# Step 2: Define the data directories and classes

data_dir = '../dataset/data1a'
train_dir = os.path.join(data_dir, 'training')
valid_dir = os.path.join(data_dir, 'validation')

# Define the classes
classes = ['No Damage', 'Damage']
damage_classes = ['00-minor', '01-moderate', '02-severe']
location_classes = ['00-front', '01-rear', '02-side']
type_classes = ['00-damage', '01-whole']


In [5]:
#Step 3: Define the data transforms

transform_train = transforms.Compose([
    transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_valid = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [6]:
#Step 4: Define the data loaders

train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
valid_dataset = datasets.ImageFolder(valid_dir, transform=transform_valid)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)


In [7]:
# Step 5: Define the model architecture

!pip install torch torchvision torchaudio
!pip install git+https://github.com/ultralytics/yolov5.git


  Cloning https://github.com/ultralytics/yolov5.git to c:\users\rasheli\appdata\local\temp\pip-req-build-kxei7xv3
  Resolved https://github.com/ultralytics/yolov5.git to commit ff6a9ac842f3a09941ac3dca5355cfa896b5f5d7


  Running command git clone -q https://github.com/ultralytics/yolov5.git 'C:\Users\Rasheli\AppData\Local\Temp\pip-req-build-kxei7xv3'
ERROR: File "setup.py" not found for legacy project git+https://github.com/ultralytics/yolov5.git.


In [8]:
import torch
import torch.nn as nn

model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5m.pt', verbose=False)



YOLOv5  2023-4-24 Python-3.9.7 torch-2.0.0+cpu CPU



requirements: C:\Users\Rasheli\.cache\torch\hub\requirements.txt not found, check failed.


Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


In [9]:
# Step 6: Define the loss function and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [10]:
#Step 7: Train the model

#Step 7: Train the model

def train_model(model, criterion, optimizer, num_epochs=10):
    num_classes = len(classes)
    for epoch in range(num_epochs):
        train_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.view(-1, num_classes, 5 + num_anchors)
            outputs = outputs.permute(0, 2, 1).contiguous()
            outputs = outputs.view(-1, num_classes)
            labels = labels.view(-1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        train_loss = train_loss / len(train_loader.dataset)
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))
num_anchors = 32  # replace 3 with the appropriate value for your use case

# def train_model(model, criterion, optimizer, num_epochs=10):
#     for epoch in range(num_epochs):
#         train_loss = 0.0
#         for inputs, labels in train_loader:
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             outputs = outputs.view(-1, num_classes, 5 + num_anchors)
#             outputs = outputs.permute(0, 2, 1).contiguous()
#             outputs = outputs.view(-1, num_classes)
#             labels = labels.view(-1)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item() * inputs.size(0)
        
#         train_loss = train_loss / len(train_loader.dataset)
#         print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))


# def train_model(model, criterion, optimizer, num_epochs=10):
#     for epoch in range(num_epochs):
#         train_loss = 0.0
#         for inputs, labels in train_loader:
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item() * inputs.size(0)
        
#         train_loss = train_loss / len(train_loader.dataset)
#         print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss))


In [11]:
train_model(model, criterion, optimizer, num_epochs=10)



RuntimeError: shape '[-1, 2, 37]' is invalid for input of size 8396640

In [ ]:
# Step 8: Save the trained model

torch.save(model.state_dict(), 'vehicle_damage_detection.pth')


In [ ]:
# Step 9: Test the model

def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuracy = correct / total
        print('Accuracy on the validation set: {:.2%}'.format(accuracy))


In [ ]:
test_model(model)


In [ ]:
#Step 10: Make predictions

def predict_image(image_path):
    image = Image.open(image_path)
    image_tensor = transform(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    output = model(input)
    index = output.data.numpy().argmax()
    return index


In [ ]:
index = predict_image('test_image.jpg')
if index == 0:
    print('The vehicle is not damaged')
else:
    print('The vehicle is damaged')


In [ ]:
def predict_image(image_path):
    image = Image.open(image_path)
    image_tensor = transform(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    output = model(input)
    _, predicted = torch.max(output.data, 1)
    return predicted.item()
